In [1]:
import albumentations as A
import cv2
from glob import glob
from ultralytics import YOLO
from roboflow import Roboflow
import os
import matplotlib.pyplot as plt

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="W3lyeuGkid3lp0RQJurI")
project = rf.workspace("project-jak2u").project("lengua-de-senas-chilena-desde-cero")
dataset = project.version(9).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.196, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to Lengua-de-señas-chilena-desde-cero-9 in yolov8:: 100%|██████████| 5298/5298 [00:01<00:00, 2968.68it/s]


In [4]:

augmentation = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.5),
])
def augment_and_save(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented = augmentation(image=image)
    augmented_image = augmented['image']
    augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
    
    dirname, filename = os.path.split(image_path)
    filename_without_extension, extension = os.path.splitext(filename)
    new_filename = f"{filename_without_extension}_changed{extension}"
    new_path = os.path.join(dirname, new_filename)
    
    cv2.imwrite(new_path, augmented_image)  
    
    return image, augmented_image  

In [5]:
train_image_paths = glob('Lengua-de-señas-chilena-desde-cero-9/train/images/*.jpg')

for i, image_path in enumerate(train_image_paths):
    '''
    if i < 5:
        original_image, transformed_image = augment_and_save(image_path)
        
        # Muestra la imagen original y la imagen transformada
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
        plt.title('Transformed Image')
        plt.show()
    '''
    augment_and_save(image_path)

In [2]:
model = YOLO("yolov8n.pt")

dataset = 'Lengua-de-señas-chilena-desde-cero-9/data.yaml'
results = model.train(data=dataset, epochs=150, imgsz=640,device=0,workers=6)


Ultralytics YOLOv8.0.196  Python-3.10.11 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=Lengua-de-seas-chilena-desde-cero-9/data.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=6, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=